In [4]:
# Importar librerías para tratamiento de datos
import pandas as pd  

# Importar librerías para web scraping y manipulación de datos
from bs4 import BeautifulSoup
import requests
import re
import random

# Importar librerías para automatización de navegadores web con Selenium
from selenium import webdriver  
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager  
from selenium.webdriver.common.keys import Keys  
from selenium.webdriver.support.ui import Select  

# Importar librerías para pausar la ejecución
from time import sleep  

# Configuraciones
pd.set_option('display.max_columns', None)  # Establece una opción de Pandas para mostrar todas las columnas de un DataFrame.

import warnings
warnings.filterwarnings('ignore')

In [5]:
df_1 = pd.read_csv("INFO_PELIS/cortos_2000_2006.csv", index_col = 0)
df_2 = pd.read_csv("INFO_PELIS/cortos_2021.csv", index_col = 0)
df_3 = pd.read_csv("INFO_PELIS/cortos_2022.csv", index_col = 0)
df_4 = pd.read_csv("INFO_PELIS/cortos_2023.csv", index_col = 0)
df_5 = pd.read_csv("INFO_PELIS/peliculas_2021_2023.csv", index_col = 0)
df_6 = pd.read_csv("INFO_PELIS/pelis_2000_2006.csv", index_col = 0)
df_7 = pd.read_csv("INFO_PELIS/pelis_2007_2010.csv", index_col = 0)
df_8 = pd.read_csv("INFO_PELIS/pelis_2011_2013.csv", index_col = 0)
df_9 = pd.read_csv("INFO_PELIS/cortos_2007_2008.csv", index_col = 0)
df_10 = pd.read_csv("INFO_PELIS/cortos_2009_2010.csv", index_col = 0)
df_11 = pd.read_csv("INFO_PELIS/cortos_2011.csv", index_col = 0)
df_12 = pd.read_csv("INFO_PELIS/cortos_2012.csv", index_col = 0)
df_13 = pd.read_csv("INFO_PELIS/cortos_2013.csv", index_col = 0)



In [6]:
df_peliculas = pd.concat([df_1,df_2,df_3,df_4,df_5,df_6,df_7,df_8,df_9,df_10,df_11,df_12,df_13], axis=0)

In [7]:
df_peliculas.head()

,Genero,Tipo,Título,Año de estreno,Mes de estreno,ID
0,Drama,Short,Chateaubriand - Cabeça de Paraíba,2000,12.0,tt0115874
1,Drama,Short,Nights,2000,3.0,tt0138655
2,Drama,Short,Caught,2000,8.0,tt0142155
3,Drama,Short,Smog,2000,3.0,tt0144613
4,Drama,Short,Safe Journey,2000,NaN,tt0188189


In [8]:
df_peliculas.tail()

,Genero,Tipo,Título,Año de estreno,Mes de estreno,ID
29062,Action,Short,Snake Eater,2013,7.0,tt28769878
29063,Action,Short,Boom Box,2013,10.0,tt28970681
29064,Action,Short,Maudit Poulet,2013,NaN,tt29279117
29065,Action,Short,I___U,2013,8.0,tt29433024
29066,Action,Short,Blind-Sided,2013,12.0,tt30310933


In [9]:
df_peliculas.to_csv('union_peliculas.csv')

In [10]:
lista_titulos= df_peliculas["Título"].to_list()

In [11]:
lista_ID = df_peliculas["ID"].to_list()

In [25]:
lista1=[df_1,df_2,df_3,df_4,df_5,df_6,df_7,df_8,df_9,df_10,df_11,df_12,df_13]

suma = 0
for e in lista1:
    suma += len(e)
print("Longitus suma de los data",suma)   
print("Longitud DATAFRAME", len(lista_ID)) 


Longitus suma de los data 312764
longitud DATAFRAME 312764


In [21]:
len(lista_ID)

312764

In [13]:
driver = webdriver.Chrome()

url = "https://www.imdb.com/?ref_=nv_home"

driver.get(url)

driver.maximize_window()

sleep(5)

# Aceptar las cookies PAG IMDB
driver.find_element("css selector", "#__next > div > div > div.sc-jrcTuL.bPmWiM > div > button.icb-btn.sc-bcXHqe.sc-dkrFOg.sc-iBYQkv.dcvrLS.ddtuHe.dRCGjd").click()

sleep(5)


## CAMBIAR A TUPLA
diccionario_peliculas = {"Puntuacion_IMDB":[], "Dirección": [],"Guionistas":[], "Argumento":[], "Duración":[], "Nombre de la película": [], "10actores":[]}
diccionario_actores = {"Actor":[], "Fehca_nacimiento":[], "Conocido_por:":[], "Que_hace":[], "Premios":[]}

lista_ID

for pelicula in lista_ID[130:133]:
    
    #Accedemos al buscador e introduccimos la peli para buscar PAG IMDB
    driver.find_element("css selector",'#suggestion-search').send_keys(pelicula, Keys.ENTER)

    sleep(random.randint(5,12))

    
# Seleccionar nombre de la pelicula
    try:
        diccionario_peliculas["Nombre de la película"].append(driver.find_element("css selector", 'span[data-testid="hero__primary-text"]').text)
    except:
        diccionario_peliculas["Nombre de la película"].append(f"Error nombre para la pelicula con id: {pelicula}")
        
# Seleccionar puntuacion 
    try: 
        diccionario_peliculas["Puntuacion_IMDB"].append(driver.find_element("css selector",'div[data-testid="hero-rating-bar__aggregate-rating__score"] > span:first-child').text)
        
    except:  
        diccionario_peliculas["Puntuacion_IMDB"].append("--")


    existe_direccion = False
    existe_guion = False

    for elemento in driver.find_elements("css selector",'li[data-testid="title-pc-principal-credit"]'):
 
        try:
            nombre = elemento.find_element("css selector", "span").text
            if nombre == "Dirección":
                existe_direccion = True

                # Seleccionar director
                try:          
                    diccionario_peliculas["Dirección"].append([e.text for e in elemento.find_elements("css selector",'div > ul > li > a') if e.text != ""])
                except:
                    diccionario_peliculas["Dirección"].append("DESCONOCIDO") 
            

            elif nombre == "Guión":
                existe_guion = True
            
                # Seleccionar Guionista
                try:          
                    diccionario_peliculas["Guionistas"].append([e.text for e in elemento.find_elements("css selector",'div > ul > li > a') if e.text != ""])
                except:
                    diccionario_peliculas["Guionistas"].append("DESCONOCIDO")
        except:
            continue

    if existe_direccion == False:    
        diccionario_peliculas["Dirección"].append("DESCONOCIDO")
    if existe_guion == False:
        diccionario_peliculas["Guionistas"].append("DESCONOCIDO")
     

    sleep(5)
    # Seleccionar ARGUMENTO
    
    try:      
        diccionario_peliculas["Argumento"].append(driver.find_element("css selector", 'span[data-testid="plot-xl"]').text)
            
    except:
        diccionario_peliculas["Argumento"].append("Desconocido")

    # Seleccionar DURACION        
    try:   
        busqueda_duracion = driver.find_element("css selector",'div.sc-491663c0-3.bdjVSf > div:first-child > ul > li:last-child').text
        if "min" in busqueda_duracion or "h" in busqueda_duracion:
            diccionario_peliculas["Duración"].append(busqueda_duracion)
        else:
            diccionario_peliculas["Duración"].append("--")
    except:
            diccionario_peliculas["Duración"].append("--")

    # Buscar los 10 primeros actores    
    try:
    
    except:

        diccionario_peliculas["10actores"].append("--")




### REVISAR:
# Lista de tuplas



In [14]:
diccionario_peliculas

{'Puntuacion_IMDB': ['7,6', '--', '--'],
 'Dirección': [['Kalman Apple'], ['Heather Dyer'], ['Stephen Winter']],
 'Guionistas': [['Kalman Apple', 'Anton Chekhov'],
  ['Heather Dyer'],
  'DESCONOCIDO'],
 'Argumento': ["Three actors and a director perform Chekhov's The Bear on a New York City bus (actually, a succession of buses, as they keep getting thrown off). The other riders on the final bus get progressively more interested.",
  '',
  ''],
 'Duración': ['28min', '--', '3min'],
 'Nombre de la película': ['Speed for Thespians', 'The Valentine', 'Untitled']}

In [51]:
driver = webdriver.Chrome()

url = "https://www.imdb.com/?ref_=nv_home"

driver.get(url)

driver.maximize_window()

sleep(5)

# Aceptar las cookies PAG IMDB
driver.find_element("css selector", "#__next > div > div > div.sc-jrcTuL.bPmWiM > div > button.icb-btn.sc-bcXHqe.sc-dkrFOg.sc-iBYQkv.dcvrLS.ddtuHe.dRCGjd").click()

sleep(5)

actores = {"actor":[]}
url_actores={"url":[]}

lista_ID

for pelicula in lista_ID[130:131]:
    
    #Accedemos al buscador e introduccimos la peli para buscar PAG IMDB
    driver.find_element("css selector",'#suggestion-search').send_keys(pelicula, Keys.ENTER)

    sleep(random.randint(5,12))

    for elemento in driver.find_elements("css selector",'section[data-testid="title-cast"]'):
        actor =[e for e in  elemento.find_elements("css selector", "div > div > div > div > a") if e.text != "" and e.text != "\n"]  
 
        for element in actor[0:10]:
            actores["actor"].append(element.text)
            url_actores["url"].append(element.get_attribute("href"))


    # try:
    
    # except:

    #     diccionario_peliculas["10actores"].append("--")

https://www.imdb.com/name/nm0364807/?ref_=tt_cl_t_1
https://www.imdb.com/name/nm0256813/?ref_=tt_cl_t_2
https://www.imdb.com/name/nm0391799/?ref_=tt_cl_t_3
https://www.imdb.com/name/nm0018916/?ref_=tt_cl_t_4
https://www.imdb.com/name/nm14869118/?ref_=tt_cl_t_5
https://www.imdb.com/name/nm1994376/?ref_=tt_cl_t_6
https://www.imdb.com/name/nm14869119/?ref_=tt_cl_t_7
https://www.imdb.com/name/nm14869117/?ref_=tt_cl_t_8
https://www.imdb.com/name/nm14869122/?ref_=tt_cl_t_9
https://www.imdb.com/name/nm14869114/?ref_=tt_cl_t_10


In [46]:
driver = webdriver.Chrome()

url = "https://www.imdb.com/?ref_=nv_home"

driver.get(url)

driver.maximize_window()

sleep(5)

# Aceptar las cookies PAG IMDB
driver.find_element("css selector", "#__next > div > div > div.sc-jrcTuL.bPmWiM > div > button.icb-btn.sc-bcXHqe.sc-dkrFOg.sc-iBYQkv.dcvrLS.ddtuHe.dRCGjd").click()

sleep(5)

actores = {"actor":[]}
url_actores={"url":[]}

lista_ID

for pelicula in lista_ID[130:131]:
    
    #Accedemos al buscador e introduccimos la peli para buscar PAG IMDB
    driver.find_element("css selector",'#suggestion-search').send_keys(pelicula, Keys.ENTER)

    for elemento in driver.find_elements("css selector",'section[data-testid="title-cast"]'):

        print(elemento.text.strip("\n"))

#         actores["actor"].append(elemento.text.strip("\n"))
#         url_actores["url"].append(elemento.get_attribute('href'))


# print(actores)
# print("_______")
# print(url_actores)

Reparto principal
Editar
Jamie Harris
Smirnov
Camilla Enders
Popova
Denis Holmes
Luka
Layla Alexander
Passenger
Olga Alexandrova
Girlfriend
Michele Ammon
Irritated Passenger
Tyler Andrews
Boyfriend
Dann R. Black
Telephone repairman
Valerie Bodoy
Homegirl
Jean Pierre Brax
Times reader
Erin Cahill
Sister(as Erin J. Cahill)
Cindy Cheung
Nurse
Justin Christopher
Skater kid
Lee Lee Core
Sipping straw girl
Jamie Elles
Goth girl
John R. Hartmann
Businessman
Betty Hudson
Friend with question lady
Stefanie Imhoff
Sister
Dirección
Kalman Apple
Guión
Kalman AppleAnton Chekhov
Todo el reparto y equipo
Producción, taquilla y más en IMDbPro


In [16]:
df_peliculas.head(25)

,Genero,Tipo,Título,Año de estreno,Mes de estreno,ID
0,Drama,Short,Chateaubriand - Cabeça de Paraíba,2000,12.0,tt0115874
1,Drama,Short,Nights,2000,3.0,tt0138655
2,Drama,Short,Caught,2000,8.0,tt0142155
3,Drama,Short,Smog,2000,3.0,tt0144613
4,Drama,Short,Safe Journey,2000,NaN,tt0188189
5,Drama,Short,Among Others,2000,6.0,tt0191782
6,Drama,Short,Ruben,2000,6.0,tt0194319
7,Drama,Short,O Pedido de Emprego,2000,2.0,tt0195979
8,Drama,Short,Soul Cages,2000,10.0,tt0202592
9,Drama,Short,Shadows,2000,3.0,tt0201919


In [18]:
driver = webdriver.Chrome()

url= "https://www.google.es/"

driver.get(url)

driver.maximize_window()

# Aceptar las cookies de GOOGLE
driver.find_element("css selector", "#L2AGLb > div").click()


sleep(3)

puntuacion_tomatoes = {"Rotten Tomatoes":[]}

lista_titulos

for pelicula in lista_titulos[0:5]:
    
    busqueda = f"puntuacion Rotten Tomatoes de la {pelicula}"


    sleep(random.randint(3,6))

    try:
        #Accedemos al buscador e introduccimos la peli para buscar PAG IMDB
        driver.find_element("css selector",'#APjFqb').send_keys(busqueda, Keys.ENTER)


        sleep(5)
        try:
            puntuacion_tomatoes["Rotten Tomatoes"].append(driver.find_element("css selector","#rso > div:nth-child(3) > div > div > div > div:nth-child(3) > div > span:nth-child(2)").text)
            driver.back()
        except:
            puntuacion_completa= (driver.find_element("css selector",'#rso > div:nth-child(1) > div > div > div:nth-child(3) > div').text)
            puntuacion = re.search(r'\b(\d+)%', puntuacion_completa)
            puntuacion_tomatoes["Rotten Tomatoes"].append(puntuacion.group())

        driver.back()
    
    except:
        puntuacion_tomatoes["Rotten Tomatoes"].append("No existe puntuacion")
    driver.back()




## MODIFICAR:
#  Se tiene que añadir al diccionario de pelis y convertir en TUPLA


In [20]:
print(puntuacion_tomatoes["Rotten Tomatoes"])

['No existe puntuacion', 'No existe puntuacion', '67%', 'No existe puntuacion', 'No existe puntuacion']


In [26]:
for id_pelicula in lista_ID[130:133]:
    
    url= f"https://www.imdb.com/title/{id_pelicula}"

    sleep(random.randint(5,12))
    llamada_pelicula = requests.get(url)
    print(llamada_pelicula.content)
    print(url)

b'<html>\r\n<head><title>403 Forbidden</title></head>\r\n<body>\r\n<center><h1>403 Forbidden</h1></center>\r\n</body>\r\n</html>\r\n'
https://www.imdb.com/title/tt0269899
b'<html>\r\n<head><title>403 Forbidden</title></head>\r\n<body>\r\n<center><h1>403 Forbidden</h1></center>\r\n</body>\r\n</html>\r\n'
https://www.imdb.com/title/tt0270026
b'<html>\r\n<head><title>403 Forbidden</title></head>\r\n<body>\r\n<center><h1>403 Forbidden</h1></center>\r\n</body>\r\n</html>\r\n'
https://www.imdb.com/title/tt0270717
